Vietnamese Traffic Sign Recognition with Deep Learning

In [ ]:
#Install YOLOv8 and Requirements
!pip install ultralytics --quiet
!pip install seaborn --quiet

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from ultralytics import YOLO
import torch
import shutil
import random


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from pathlib import Path

# Set dataset path
dataset_path = "D:/2025/Vietnamese-Traffic-Sign/myData"

# Get list of classes and count images
classes = sorted(os.listdir(dataset_path))
class_counts = {}

for class_name in classes:
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        num_images = len(os.listdir(class_path))
        class_counts[class_name] = num_images

# Print dataset summary
print(f"Total number of classes: {len(classes)}")
print("\nImages per class:")
for class_name, count in class_counts.items():
    print(f"Class {class_name}: {count} images")

total_images = sum(class_counts.values())
print(f"\nTotal number of images: {total_images}")

📊 Total number of classes: 25

🔍 Images per class:
Class 0: 451 images
Class 1: 4920 images
Class 10: 2490 images
Class 11: 1470 images
Class 12: 2100 images
Class 13: 510 images
Class 14: 2850 images
Class 15: 1140 images
Class 16: 1020 images
Class 17: 450 images
Class 18: 1288 images
Class 19: 780 images
Class 2: 2610 images
Class 20: 2280 images
Class 21: 720 images
Class 22: 390 images
Class 23: 660 images
Class 24: 450 images
Class 3: 2670 images
Class 4: 3750 images
Class 5: 3510 images
Class 6: 780 images
Class 7: 2730 images
Class 8: 2670 images
Class 9: 2790 images

📈 Total number of images: 45479


In [ ]:
#Chia train/val/test 80/10/10
os.makedirs("dataset/train", exist_ok=True)
os.makedirs("dataset/val", exist_ok=True)
os.makedirs("dataset/test", exist_ok=True)

for cls in classes:
    img_dir = os.path.join(dataset_path, cls)
    imgs = os.listdir(img_dir)
    random.shuffle(imgs)

    n = len(imgs)
    n_train = int(0.8 * n)
    n_val = int(0.1 * n)

    splits = {
        "train": imgs[:n_train],
        "val": imgs[n_train:n_train+n_val],
        "test": imgs[n_train+n_val:]
    }

    for split, img_list in splits.items():
        os.makedirs(f"dataset/{split}/{cls}", exist_ok=True)
        for img in img_list:
            shutil.copy(os.path.join(img_dir, img), f"dataset/{split}/{cls}/{img}")


In [ ]:
#Tạo YAML config file cho YOLO
# ===================================
with open("traffic.yaml", "w") as f:
    f.write(f"""
path: dataset
train: train
val: val
test: test
names: {classes}
""")


In [ ]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
from ultralytics.utils import LOGGER

# ======= Cấu hình =======
YAML_DATA = "D:/2025/Yolo_Code/dataset"     # file data (train/val/test)
EPOCHS    = 50
IMGSZ     = 224
BATCH     = 32
WORKERS   = 8
DEVICE_ID = 0 if torch.cuda.is_available() else None  # None -> CPU

# Hàm lấy top1 cho mọi version Ultralytics
def _get_top1(mets):
    for k in ("top1", "accuracy_top1"):
        if hasattr(mets, k):
            return float(getattr(mets, k))
    rd = getattr(mets, "results_dict", None)
    if isinstance(rd, dict):
        for k in ("top1", "accuracy_top1"):
            if k in rd:
                return float(rd[k])
    raise RuntimeError("Không lấy được top1 từ metrics.")

# ======= Load model và gắn callback =======
model = YOLO("yolov8m-cls.pt")

def on_fit_epoch_end(trainer):
    
    try:
        save_dir   = Path(trainer.save_dir)
        weightsdir = save_dir / "weights"
        last_ckpt  = weightsdir / "last.pt"
        if not last_ckpt.exists():
            LOGGER.warning(f"[callback] Chưa thấy {last_ckpt}, bỏ qua epoch này.")
            return

        # Đánh giá train bằng evaluator trên last.pt
        m_wrap = YOLO(str(last_ckpt))
        mets = m_wrap.val(
            data=YAML_DATA, split="train",
            imgsz=IMGSZ, batch=BATCH,
            device=(trainer.device if hasattr(trainer, "device") else DEVICE_ID),
            verbose=False
        )
        train_acc = _get_top1(mets)

        # Ghi vào results.csv
        results_csv = save_dir / "results.csv"
        df = pd.read_csv(results_csv)
        if "train/acc" not in df.columns:
            df["train/acc"] = pd.NA
        row_idx = min(trainer.epoch, len(df)-1)  # phòng lệch
        df.loc[row_idx, "train/acc"] = train_acc
        df.to_csv(results_csv, index=False)

        LOGGER.info(f"Epoch {trainer.epoch+1}/{trainer.epochs} | "
                    f"Train Acc: {train_acc:.4f} | "
                    f"Val Acc: {float(df.loc[row_idx, 'metrics/accuracy_top1']):.4f} | "
                    f"Train Loss: {float(df.loc[row_idx, 'train/loss']):.4f} | "
                    f"Val Loss: {float(df.loc[row_idx, 'val/loss']):.4f}")
    except Exception as e:
        LOGGER.warning(f"[callback] lỗi log train acc: {e}")

# Gắn callback
model.add_callback("on_fit_epoch_end", on_fit_epoch_end)

# ======= Train =======
train_results = model.train(
    data=YAML_DATA,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    workers=WORKERS,
    device=DEVICE_ID,
    name="train_with_last_eval",   # tên run
    project=None                   # mặc định runs/classify/...
)

# ======= đọc CSV & Vẽ biểu đồ (Loss & Acc cho Train/Val) =======
save_dir    = Path(getattr(model, "trainer").save_dir)
results_csv = save_dir / "results.csv"

df = pd.read_csv(results_csv).copy()
if "train/acc" not in df.columns:
    
    df["train/acc"] = pd.NA

# Lưu CSV thêm cột train/acc
merged_csv = save_dir / "results_with_train_acc.csv"
df.to_csv(merged_csv, index=False)
print(f"Saved CSV: {merged_csv}")

# Vẽ biểu đồ (Loss & Acc cho Train/Val)
fig, ax1 = plt.subplots(figsize=(12,7))
ax2 = ax1.twinx()
l1, = ax1.plot(df.index+1, df["train/loss"], label="Train Loss")
l2, = ax1.plot(df.index+1, df["val/loss"],   label="Val Loss")
ax1.set_xlabel("Epoch"); ax1.set_ylabel("Loss"); ax1.grid(True, alpha=0.3)
l3, = ax2.plot(df.index+1, df["train/acc"].astype(float), label="Train Acc", linestyle="--")
l4, = ax2.plot(df.index+1, df["metrics/accuracy_top1"],   label="Val Acc")
ax2.set_ylabel("Accuracy")
ax1.legend([l1,l2,l3,l4], ["Train Loss","Val Loss","Train Acc","Val Acc"], loc="center right")
plt.title("YOLOv8 Classification – Train/Val Loss & Acc (per-epoch via last.pt evaluator)")
plt.tight_layout()
out_png = save_dir / "metrics_train_val_4curves.png"
plt.savefig(out_png, dpi=150); plt.show()
print(f"Saved plot: {out_png}")


New https://pypi.org/project/ultralytics/8.3.234 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.226  Python-3.13.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/2025/Yolo_Code/dataset, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_with_last_

<Figure size 1200x700 with 2 Axes>

🖼️ Saved plot: D:\2025\Yolo_Code\runs\classify\train_with_last_eval3\metrics_train_val_4curves.png


In [ ]:
import pandas as pd

# Path file CSV
csv_path = 'D:/2025/Yolo_Code/runs/classify/train_with_last_eval3/results_with_train_acc.csv'

# Đọc file CSV
df = pd.read_csv(csv_path)

# Kiểm tra các cột cần thiết
required_cols = ['epoch', 'train/loss', 'metrics/accuracy_top1', 'val/loss', 'train/acc']
if all(col in df.columns for col in required_cols):
    for _, row in df.iterrows():
        print(f"Epoch {int(row['epoch'])}/50, Train Loss: {row['train/loss']:.4f}, "
              f"Train Acc: {row['train/acc']:.4f}, Val Loss: {row['val/loss']:.4f} ,Val Acc: {row['metrics/accuracy_top1']:.4f}")
else:
    print("Không đủ các cột cần thiết trong file CSV.")

Epoch 1/50, Train Loss: 2.0840, Train Acc: 0.8179, Val Loss: 0.6329 ,Val Acc: 0.8130
Epoch 2/50, Train Loss: 0.4645, Train Acc: 0.9625, Val Loss: 0.1292 ,Val Acc: 0.9593
Epoch 3/50, Train Loss: 0.2464, Train Acc: 0.9753, Val Loss: 0.0751 ,Val Acc: 0.9736
Epoch 4/50, Train Loss: 0.1744, Train Acc: 0.9878, Val Loss: 0.0415 ,Val Acc: 0.9846
Epoch 5/50, Train Loss: 0.1338, Train Acc: 0.9927, Val Loss: 0.0254 ,Val Acc: 0.9916
Epoch 6/50, Train Loss: 0.1126, Train Acc: 0.9958, Val Loss: 0.0191 ,Val Acc: 0.9923
Epoch 7/50, Train Loss: 0.1016, Train Acc: 0.9974, Val Loss: 0.0162 ,Val Acc: 0.9943
Epoch 8/50, Train Loss: 0.0917, Train Acc: 0.9977, Val Loss: 0.0118 ,Val Acc: 0.9958
Epoch 9/50, Train Loss: 0.0852, Train Acc: 0.9986, Val Loss: 0.0106 ,Val Acc: 0.9967
Epoch 10/50, Train Loss: 0.0842, Train Acc: 0.9989, Val Loss: 0.0085 ,Val Acc: 0.9965
Epoch 11/50, Train Loss: 0.0752, Train Acc: 0.9993, Val Loss: 0.0076 ,Val Acc: 0.9969
Epoch 12/50, Train Loss: 0.0728, Train Acc: 0.9995, Val Loss: 0

In [ ]:
#Load YOLOv8 Model
model = YOLO('yolov8m-cls.pt')

In [ ]:
# 🧪 STEP 6: Validation
metrics = model.val()
print("Evaluation Metrics:", metrics)

Ultralytics 8.3.226  Python-3.13.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,794,681 parameters, 0 gradients, 41.7 GFLOPs
train: D:\2025\Yolo_Code\dataset\train... found 36382 images in 25 classes  
val: D:\2025\Yolo_Code\dataset\val... found 4547 images in 25 classes  
test: D:\2025\Yolo_Code\dataset\test... found 4550 images in 25 classes  
val: Fast image access  (ping: 0.00.0 ms, read: 3.21.8 MB/s, size: 0.9 KB)
val: Scanning D:\2025\Yolo_Code\dataset\val... 4546 images, 1 corrupt: 100% ━━━━━━━━━━━━ 4546/4546 7.5Mit/s 0.0s0s
val: D:\2025\Yolo_Code\dataset\val\14\00047_00014.jpg: ignoring corrupt image/label: [Errno 22] Invalid argument
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 285/285 52.3it/s 5.5s0.1s
                   all          1          1
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to D:\2025\Yolo_Code\runs\classify\val
🔍 Evaluati

In [20]:
from sklearn.metrics import accuracy_score

# Ensure y_true / y_pred exist. If not, compute them using the test folder (same logic as CELL INDEX 18).
if 'y_true' not in globals() or len(y_true) == 0 or 'y_pred' not in globals() or len(y_pred) == 0:
	y_true, y_pred = [], []
	val_dir = "D:/2025/Yolo_Code/dataset/test"
	classes = sorted(os.listdir(val_dir))

	for class_index, class_name in enumerate(classes):
		class_folder = os.path.join(val_dir, class_name)
		if not os.path.isdir(class_folder):
			continue
		for img_file in os.listdir(class_folder):
			img_path = os.path.join(class_folder, img_file)
			try:
				results = model(img_path, verbose=False)
				if len(results) == 0:
					continue
				res = results[0]
				
				# Get probs robustly
				probs = getattr(res, 'probs', None)
				if probs is None:
					continue

				# Extract the top1 prediction index
				top1_idx = getattr(probs, 'top1', None)
				if top1_idx is None:
					continue
				
				pred = int(top1_idx)
				y_pred.append(pred)
				y_true.append(class_index)
			except Exception as e:
				# skip images that cause errors
				continue

# Validate lists
if len(y_true) == 0:
	raise RuntimeError("No predictions collected. Ensure the test dataset exists and the model is loaded (see CELL INDEX 18/20).")

# Compute and print accuracy
acc = accuracy_score(y_true, y_pred)
correct = sum(1 for a, b in zip(y_true, y_pred) if a == b)
print(f" Test Accuracy: {acc:.4f} ({correct}/{len(y_true)})")


 Test Accuracy: 0.9996 (4548/4550)


In [ ]:
# Classification Report & Confusion Matrix
val_dir = "D:/2025/Yolo_Code/dataset/test"
classes = sorted(os.listdir(val_dir))
y_true, y_pred = [], []

for class_index, class_name in enumerate(classes):
    class_folder = os.path.join(val_dir, class_name)
    for img_file in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_file)
        try:
            results = model(img_path, verbose=False)[0]
            pred_class = results.probs.top1
            y_pred.append(pred_class)
            y_true.append(class_index)
        except:
            continue

report = classification_report(y_true, y_pred, target_names=classes, digits=4)
print("Classification Report:\n", report)

os.makedirs("outputs", exist_ok=True)
with open("outputs/yolov8_classification_report.txt", "w") as f:
    f.write(report)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig("outputs/yolov8_confusion_matrix.png")
plt.show()

📋 Classification Report:
               precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        46
           1     1.0000    1.0000    1.0000       492
          10     1.0000    1.0000    1.0000       249
          11     1.0000    1.0000    1.0000       147
          12     1.0000    0.9905    0.9952       210
          13     1.0000    1.0000    1.0000        51
          14     1.0000    1.0000    1.0000       285
          15     1.0000    1.0000    1.0000       114
          16     1.0000    1.0000    1.0000       102
          17     0.9783    1.0000    0.9890        45
          18     1.0000    1.0000    1.0000       130
          19     1.0000    1.0000    1.0000        78
           2     1.0000    1.0000    1.0000       261
          20     1.0000    1.0000    1.0000       228
          21     1.0000    1.0000    1.0000        72
          22     1.0000    1.0000    1.0000        39
          23     1.0000    1.0000    1.0000        66
 

<Figure size 1200x1000 with 2 Axes>

In [ ]:
# FPS Measurement
test_images = [os.path.join(val_dir, classes[0], f) for f in os.listdir(os.path.join(val_dir, classes[0]))[:20]]
start_time = time.time()
for img in test_images:
    model(img, verbose=False)
end_time = time.time()
fps = len(test_images) / (end_time - start_time)
print(f"⚡ Inference FPS: {fps:.2f} frames/sec")

⚡ Inference FPS: 27.02 frames/sec


API Streamlit

In [ ]:
# API test: YOLOv8 classification model

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics import YOLO
import pandas as pd
import torch

# --- User-editable params ---
dataset_path = "D:/2025/Vietnamese-Traffic-Sign/myData"
labels_csv_path = 'D:/2025/Vietnamese-Traffic-Sign/labels.csv'
possible_weights = [
    'D:/2025/Yolo_Code/runs/yolov8_cls_vi_signs_gpu/weights/best.pt',
    'D:/2025/Yolo_Code/runs/yolov8_cls_vi_signs_gpu/weights/last.pt',
    'yolov8m-cls.pt',
]
test_image_path = 'D:/2025/Test_Anh/50.jpg'
TOP_K = 5

# --- Resolve class names from dataset folder if possible ---
if os.path.isdir(dataset_path):
    class_names = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])
else:
    class_names = None

print(f"Dataset path: {dataset_path}\nFound {len(class_names) if class_names else 0} class folders")

# --- Load labels.csv if available and build mapping ---
class_id_to_display = {}
if os.path.exists(labels_csv_path):
    try:
        labels_df = pd.read_csv(labels_csv_path)
        print(f"Loaded labels CSV with columns: {labels_df.columns.tolist()}")
        id_col_candidates = ['ClassID', 'class_id', 'ID', 'id', labels_df.columns[0]]
        name_col_candidates = ['Name', 'name', 'Label', 'label', labels_df.columns[1] if labels_df.shape[1] > 1 else labels_df.columns[-1]]
        id_col = None
        name_col = None
        for c in id_col_candidates:
            if c in labels_df.columns:
                id_col = c
                break
        for c in name_col_candidates:
            if c in labels_df.columns:
                name_col = c
                break
        if id_col is None:
            id_col = labels_df.columns[0]
        if name_col is None and labels_df.shape[1] > 1:
            name_col = labels_df.columns[1]
        if name_col is None:
            name_col = labels_df.columns[0]
        print(f"Using id column: '{id_col}', name column: '{name_col}'")
        for _, row in labels_df.iterrows():
            raw_id = row[id_col]
            display = str(row[name_col])
            try:
                key_int = int(raw_id)
                class_id_to_display[key_int] = display
            except Exception:
                pass
            key_str = str(raw_id)
            class_id_to_display[key_str] = display
        print(f"Built display mapping for {len(class_id_to_display)} entries (mixed int/str keys)")
    except Exception as e:
        print(f"Failed to load/parse labels CSV: {e}")
else:
    print(f"Labels CSV not found at {labels_csv_path} — will attempt to use dataset folder names or model.names")

# --- Find a weights file to load ---
weights_path = None
for p in possible_weights:
    if os.path.exists(p):
        weights_path = p
        break

if weights_path is None:
    print("No trained weights file found in the standard locations. Falling back to 'yolov8m-cls.pt' (downloaded model) if installed.")
    weights_path = 'yolov8m-cls.pt'

print(f"Using weights: {weights_path}")

# --- Load model ---
try:
    model = YOLO(weights_path)
except Exception as e:
    print("Failed to load model:", e)
    raise

# If class names are not available from dataset, try to get from model.names
if not class_names:
    try:
        model_names = model.names
        if isinstance(model_names, dict):
            class_names = [model_names[i] for i in sorted(model_names.keys())]
        else:
            class_names = list(model_names)
        print(f"Loaded {len(class_names)} class names from model")
    except Exception:
        class_names = None

if class_names is None:
    print("Warning: Could not determine class folder names. Predictions will show index or model names.")

# --- Helper: map predicted index or folder name to display name ---
def get_display_name(pred_index:int, folder_name: str=None):
    if pred_index in class_id_to_display:
        return class_id_to_display[pred_index]
    if folder_name is not None:
        if folder_name in class_id_to_display:
            return class_id_to_display[folder_name]
        try:
            k = int(folder_name)
            if k in class_id_to_display:
                return class_id_to_display[k]
        except Exception:
            pass
    try:
        if hasattr(model, 'names'):
            mn = model.names
            if isinstance(mn, dict) and pred_index in mn:
                return str(mn[pred_index])
            elif isinstance(mn, (list, tuple)) and 0 <= pred_index < len(mn):
                return str(mn[pred_index])
    except Exception:
        pass
    if folder_name is not None:
        return str(folder_name)
    return str(pred_index)

# --- Check test image ---
if not os.path.exists(test_image_path):
    raise FileNotFoundError(f"Test image not found: {test_image_path}")

print(f"Running inference on: {test_image_path}")

# --- Run inference ---
results = model(test_image_path, imgsz=224)  # adjust imgsz if you used different size during train
if len(results) == 0:
    raise RuntimeError("No results returned from model inference")

res = results[0]

# Get probs robustly (Ultralytics provides .probs for classification models)
probs_raw = getattr(res, 'probs', None)
if probs_raw is None:
    for alt in ('probs', 'scores', 'conf', 'data'):
        probs_raw = getattr(res, alt, None)
        if probs_raw is not None:
            break

if probs_raw is None:
    raise RuntimeError('No probabilities found in result object (no .probs/.scores/.conf)')

# --- Robust conversion: try many extraction strategies and provide good debug on failure ---
try:
    import numpy as _np

    def _is_scalar_like(x):
        return isinstance(x, (float, int, _np.floating, _np.integer))

    def to_number(x):
        """Attempt to extract a single float from x, recursively."""
        if isinstance(x, torch.Tensor):
            return float(x.detach().cpu().item())
        if _is_scalar_like(x):
            return float(x)
        if hasattr(x, "item") and not callable(x):
            try:
                return float(x.item())
            except Exception:
                pass
        for attr in ("tensor", "probs", "scores", "data", "values", "value", "score", "prob", "confidence", "conf"):
            if hasattr(x, attr):
                v = getattr(x, attr)
                v = v() if callable(v) else v
                return to_number(v)
        if hasattr(x, "numpy"):
            try:
                arr = x.numpy()
                if _is_scalar_like(arr):
                    return float(arr)
                if hasattr(arr, "ravel"):
                    flat = arr.ravel()
                    if flat.size == 1:
                        return float(flat[0])
            except Exception:
                pass
        if hasattr(x, "tolist"):
            try:
                t = x.tolist()
                if _is_scalar_like(t):
                    return float(t)
                if isinstance(t, (list, tuple)) and len(t) == 1:
                    return to_number(t[0])
            except Exception:
                pass
        raise ValueError(f"Cannot convert object of type {type(x)} to a float")

    def convert_to_numpy(obj):
        """Convert obj into a 1-D numpy array of floats."""
        if isinstance(obj, torch.Tensor):
            return obj.detach().cpu().numpy()
        if isinstance(obj, _np.ndarray):
            return obj
        if _is_scalar_like(obj):
            return _np.asarray([float(obj)])
        if hasattr(obj, "numpy"):
            try:
                arr = obj.numpy()
                return _np.asarray(arr)
            except Exception:
                pass
        if hasattr(obj, "tolist"):
            try:
                t = obj.tolist()
                return convert_to_numpy(t)
            except Exception:
                pass
        try:
            seq = list(obj)
        except Exception as e:
            raise RuntimeError(f"Object of type {type(obj)} is not iterable and not convertible: {e}")

        vals = []
        for i, e in enumerate(seq):
            try:
                n = to_number(e)
                vals.append(n)
                continue
            except Exception:
                pass
            try:
                a = convert_to_numpy(e)
                a = _np.asarray(a).ravel()
                vals.extend([float(x) for x in a.tolist()])
            except Exception as ee:
                raise RuntimeError(f"Failed to extract numeric value from element #{i} of type {type(e)}: {ee}")

        return _np.asarray(vals)

    try:
        probs = convert_to_numpy(probs_raw)
    except Exception as e:
        type_info = f"type(probs_raw)={type(probs_raw)}"
        try:
            repr_snip = repr(probs_raw)
            if len(repr_snip) > 800:
                repr_snip = repr_snip[:800] + '...'
        except Exception:
            repr_snip = '<unreprable>'
        raise RuntimeError(f"Could not convert probs to numpy array: {e}\n{type_info}\nrepr(probs_raw)={repr_snip}")

except Exception as e:
    raise RuntimeError(f'Could not convert probs to numpy array: {e}')

probs = np.asarray(probs)
if probs.ndim == 2 and probs.shape[0] == 1:
    probs = probs[0]

if probs.size == 0:
    raise RuntimeError('Probabilities array is empty after conversion')

num_classes = probs.shape[0]
print(f"Number of classes (model output): {num_classes}")

# --- Get top-K ---
topk = min(TOP_K, num_classes)
top_indices = np.argsort(probs)[::-1][:topk]
top_probs = probs[top_indices]

# Print top-K using display names
print('\nTop predictions:')
for rank, (i, p) in enumerate(zip(top_indices, top_probs), start=1):
    idx = int(i)
    folder_name = None
    if class_names and 0 <= idx < len(class_names):
        folder_name = class_names[idx]
    display_name = get_display_name(idx, folder_name)
    print(f"{rank}. {display_name} (Index {idx}) — Confidence: {p:.4f} ({p*100:.2f}%)")

# --- Show image with top-1 display name in title ---
img = Image.open(test_image_path).convert('RGB')
plt.figure(figsize=(8,6))
plt.imshow(img)
best_idx = int(top_indices[0])
best_folder = class_names[best_idx] if class_names and 0 <= best_idx < len(class_names) else None
best_name = get_display_name(best_idx, best_folder)
best_conf = float(top_probs[0])
plt.title(f"Predicted: {best_name} — Confidence: {best_conf:.4f}", fontsize=12)
plt.axis('off')
plt.show()

# Print API-style output
prediction = {
    'image': os.path.basename(test_image_path),
    'top1': {'index': int(best_idx), 'folder_name': best_folder, 'display_name': best_name, 'confidence': best_conf},
    'topk': [ {'index': int(i), 'folder_name': (class_names[int(i)] if class_names and 0 <= int(i) < len(class_names) else None), 'display_name': get_display_name(int(i), (class_names[int(i)] if class_names and 0 <= int(i) < len(class_names) else None)), 'confidence': float(p)} for i,p in zip(top_indices, top_probs) ]
}

print('\nAPI-style output (dict):')
print(prediction)


Dataset path: D:/2025/Vietnamese-Traffic-Sign/myData
Found 25 class folders
Loaded labels CSV with columns: ['ClassId', 'Name']
Using id column: 'ClassId', name column: 'Name'
Built display mapping for 50 entries (mixed int/str keys)
Using weights: D:/2025/Yolo_Code/runs/yolov8_cls_vi_signs_gpu/weights/best.pt
Running inference on: D:/2025/Test_Anh/50.jpg

image 1/1 D:\2025\Test_Anh\50.jpg: 224x224 2 1.00, 8 0.00, 7 0.00, 21 0.00, 0 0.00, 58.4ms
Speed: 5.0ms preprocess, 58.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Number of classes (model output): 25

Top predictions:
1. Bien bao cam nguoc chieu (Index 12) — Confidence: 0.9998 (99.98%)
2. Bien bao giao nhau vong xuyen (Index 23) — Confidence: 0.0000 (0.00%)
3. Bien bao di thang hoac re trai (Index 22) — Confidence: 0.0000 (0.00%)
4. Bien bao dang bao tri duong bo (Index 14) — Confidence: 0.0000 (0.00%)
5. Bien bao gioi han toc do 20kmh (Index 0) — Confidence: 0.0000 (0.00%)

API-style output (dict):
{'image':